<a href="https://colab.research.google.com/github/joppii/watermark/blob/main/train_watermark_removal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 AI Watermark Removal - Training on Google Colab

このノートブックでは、Google Colaboratoryでウォーターマーク除去モデルをGPUで高速学習できます。

## 📋 実行前の準備

1. **GPU を有効化**: ランタイム → ランタイムのタイプを変更 → T4 GPU を選択
2. **Google Drive をマウント**: 学習データとモデルの保存用
3. **学習データの準備**: Drive に画像をアップロード

## 🔧 Step 1: 環境セットアップ

In [ ]:
# Google Drive をマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# リポジトリをクローン
!git clone https://github.com/joppii/watermark.git
%cd watermark

In [ ]:
# 依存関係をインストール
!pip install -q torch torchvision numpy opencv-python Pillow scikit-image scipy tqdm tensorboard PyYAML colorama

In [ ]:
# GPU確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 📁 Step 2: データ準備

### オプション A: Google Drive から画像を使用

In [ ]:
# データディレクトリの設定
import os

# Google Drive内のパスを指定（あなたの環境に合わせて変更）
DRIVE_DATA_DIR = "/content/drive/MyDrive/watermark_data"
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/watermark_models"

# ローカル作業ディレクトリ
LOCAL_DATA_DIR = "/content/watermark/data"

# ディレクトリ作成
!mkdir -p {DRIVE_DATA_DIR}/original
!mkdir -p {DRIVE_DATA_DIR}/watermarks
!mkdir -p {DRIVE_OUTPUT_DIR}
!mkdir -p {LOCAL_DATA_DIR}

print("✓ ディレクトリ作成完了")
print(f"\n📂 Google Drive データフォルダ: {DRIVE_DATA_DIR}")
print(f"💾 モデル保存先: {DRIVE_OUTPUT_DIR}")
print(f"\n⚠️ {DRIVE_DATA_DIR}/original/ に学習用のクリーン画像をアップロードしてください")

### オプション B: サンプルデータセットをダウンロード（テスト用）

In [ ]:
# サンプル画像をダウンロード（テスト用）
# 実際の学習には、自分の画像を使用することを推奨

!mkdir -p data/original

# 例: Unsplash から画像をダウンロード（実際には自分のデータセットを使用）
# !wget -O data/original/sample1.jpg "https://source.unsplash.com/random/800x600?nature"
# !wget -O data/original/sample2.jpg "https://source.unsplash.com/random/800x600?city"
# ... さらに画像を追加

print("⚠️ このセルはコメントアウトされています")
print("実際には、Google Drive に画像をアップロードするか、")
print("データセットをダウンロードするコマンドを追加してください")

## 🎨 Step 3: ウォーターマーク作成

In [ ]:
# テキストウォーターマークを作成
!python src/prepare_data.py text \
  -t "SAMPLE" \
  -o {DRIVE_DATA_DIR}/watermarks/sample \
  -n 20

!python src/prepare_data.py text \
  -t "COPYRIGHT" \
  -o {DRIVE_DATA_DIR}/watermarks/copyright \
  -n 20

!python src/prepare_data.py text \
  -t "WATERMARK" \
  -o {DRIVE_DATA_DIR}/watermarks/watermark \
  -n 20

In [ ]:
# 作成されたウォーターマークを確認
import glob
from PIL import Image
import matplotlib.pyplot as plt

watermark_files = glob.glob(f"{DRIVE_DATA_DIR}/watermarks/*/*.png")[:5]

if watermark_files:
    fig, axes = plt.subplots(1, len(watermark_files), figsize=(15, 3))
    for idx, wm_file in enumerate(watermark_files):
        img = Image.open(wm_file)
        if len(watermark_files) == 1:
            axes.imshow(img)
            axes.set_title(f"Watermark {idx+1}")
            axes.axis('off')
        else:
            axes[idx].imshow(img)
            axes[idx].set_title(f"Watermark {idx+1}")
            axes[idx].axis('off')
    plt.tight_layout()
    plt.show()
    print(f"✓ {len(glob.glob(f'{DRIVE_DATA_DIR}/watermarks/*/*.png'))} 個のウォーターマークを作成")
else:
    print("⚠️ ウォーターマークが見つかりません")

## 🔄 Step 4: 学習データセット生成

In [ ]:
# 合成ウォーターマークデータセットを作成
!python src/prepare_data.py synthetic \
  -i {DRIVE_DATA_DIR}/original \
  --output-clean {LOCAL_DATA_DIR}/train/clean \
  --output-watermarked {LOCAL_DATA_DIR}/train/watermarked \
  -w {DRIVE_DATA_DIR}/watermarks/*/*.png \
  -n 3

print("\n✓ データセット生成完了")

In [ ]:
# データセットのプレビュー
import cv2
import numpy as np

clean_files = sorted(glob.glob(f"{LOCAL_DATA_DIR}/train/clean/*.png"))[:3]
watermarked_files = sorted(glob.glob(f"{LOCAL_DATA_DIR}/train/watermarked/*.png"))[:3]

if clean_files and watermarked_files:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    for idx in range(min(3, len(clean_files))):
        # Clean image
        clean_img = cv2.imread(clean_files[idx])
        clean_img = cv2.cvtColor(clean_img, cv2.COLOR_BGR2RGB)
        axes[0, idx].imshow(clean_img)
        axes[0, idx].set_title(f"Clean {idx+1}")
        axes[0, idx].axis('off')
        
        # Watermarked image
        wm_img = cv2.imread(watermarked_files[idx])
        wm_img = cv2.cvtColor(wm_img, cv2.COLOR_BGR2RGB)
        axes[1, idx].imshow(wm_img)
        axes[1, idx].set_title(f"Watermarked {idx+1}")
        axes[1, idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 データセット統計:")
    print(f"クリーン画像: {len(glob.glob(f'{LOCAL_DATA_DIR}/train/clean/*.png'))} 枚")
    print(f"ウォーターマーク付き画像: {len(glob.glob(f'{LOCAL_DATA_DIR}/train/watermarked/*.png'))} 枚")
else:
    print("⚠️ データが見つかりません。Step 4を再実行してください。")

## 🏋️ Step 5: モデル学習

### 学習パラメータの設定

In [ ]:
# 学習パラメータ
EPOCHS = 50          # エポック数（テスト: 20-50, 本番: 100-200）
BATCH_SIZE = 8       # バッチサイズ（GPU T4: 8-16が推奨）
LEARNING_RATE = 0.0001

print(f"学習設定:")
print(f"  エポック数: {EPOCHS}")
print(f"  バッチサイズ: {BATCH_SIZE}")
print(f"  学習率: {LEARNING_RATE}")

In [ ]:
# 学習開始
!python src/train.py \
  --clean-dir {LOCAL_DATA_DIR}/train/clean \
  --watermarked-dir {LOCAL_DATA_DIR}/train/watermarked \
  --epochs {EPOCHS} \
  --batch-size {BATCH_SIZE} \
  --val-split 0.1

## 📊 Step 6: TensorBoard で学習状況を確認

In [ ]:
# TensorBoard を起動
%load_ext tensorboard
%tensorboard --logdir runs/watermark_removal

## 💾 Step 7: モデルを Google Drive に保存

In [ ]:
# 学習済みモデルを Google Drive にコピー
import shutil
from pathlib import Path

model_dir = Path("models/pretrained")
if model_dir.exists():
    # 最新のチェックポイントを取得
    checkpoints = sorted(model_dir.glob("*.pth"))
    
    if checkpoints:
        for checkpoint in checkpoints:
            dest = Path(DRIVE_OUTPUT_DIR) / checkpoint.name
            shutil.copy(checkpoint, dest)
            print(f"✓ {checkpoint.name} → {dest}")
        
        print(f"\n✅ {len(checkpoints)} 個のモデルを Google Drive に保存しました")
        print(f"保存先: {DRIVE_OUTPUT_DIR}")
    else:
        print("⚠️ チェックポイントが見つかりません")
else:
    print("⚠️ models/pretrained ディレクトリが見つかりません")

## 🧪 Step 8: 学習済みモデルのテスト

In [ ]:
# テスト画像で推論
test_image = sorted(glob.glob(f"{LOCAL_DATA_DIR}/train/watermarked/*.png"))[0]
best_model = sorted(glob.glob("models/pretrained/*_best.pth"))

if best_model:
    best_model = best_model[-1]
    print(f"使用モデル: {best_model}")
    
    !python src/main.py \
      -i {test_image} \
      -o output/test_result.png \
      --method ai \
      --model {best_model} \
      --save-comparison
    
    # 結果を表示
    if Path("output/test_result_comparison.png").exists():
        result_img = Image.open("output/test_result_comparison.png")
        plt.figure(figsize=(15, 8))
        plt.imshow(result_img)
        plt.axis('off')
        plt.title("左: オリジナル | 右: ウォーターマーク除去後")
        plt.show()
else:
    print("⚠️ 学習済みモデルが見つかりません。Step 5を実行してください。")

## 📥 Step 9: モデルをローカルにダウンロード（オプション）

In [ ]:
# モデルファイルを圧縮
!zip -r trained_models.zip models/pretrained/*.pth

print("✓ モデルを trained_models.zip に圧縮しました")
print("左のファイルブラウザからダウンロードできます")

# または Google Drive から直接ダウンロード可能
print(f"\nまたは Google Drive から: {DRIVE_OUTPUT_DIR}")

## 🎉 完了！

### 次のステップ

1. **モデルをダウンロード**: Google Drive または上記の zip ファイルから
2. **ローカルで使用**: 
   ```bash
   python src/main.py \
     -i input.png \
     -o output.png \
     --method ai \
     --model path/to/your/model.pth \
     --auto-detect
   ```

### 学習結果が良くない場合

- ✅ より多くのデータで学習（推奨: 2000枚以上）
- ✅ エポック数を増やす（100-200）
- ✅ より多様なウォーターマークを使用
- ✅ データ品質を確認

### 学習を中断した場合

チェックポイントから再開できます:
```python
!python src/train.py \
  --clean-dir {LOCAL_DATA_DIR}/train/clean \
  --watermarked-dir {LOCAL_DATA_DIR}/train/watermarked \
  --resume models/pretrained/checkpoint_epoch_XX.pth \
  --epochs 100
```

詳細は [TRAINING.md](https://github.com/joppii/watermark/blob/main/TRAINING.md) を参照してください。